In [1]:
import os
import sys
import time
import numpy as np
import laspy
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from pyproj import CRS
from osgeo import gdal
import netCDF4 as nc
from laspy.file import File
from laspy.header import LasHeader
from bson import ObjectId
# sys.path.append("..")
from parser.parser_3dtiles.parser_3dtiles import Parser3DTiles
from parser.parser_relief import ParserRelief
from parser.parser_physicalfield import ParserPhysicalField
from parser.parser_pointcloud import ParserPointcloud
from data_operations.query import Query
from data_operations.remove import Remove
from data_operations.update import Update
from pathlib import Path
from py3dtiles.tileset.tileset import TileSet
from pyhdf.SD import SD, SDC
import pprint
import h5py
import pandas as pd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import gdal2tiles
from osgeo import gdal

## 0 数据入库

In [2]:
def upload():

    pid = os.getpid() # 获取当前进程的进程ID（Process ID）
    available_cores = [i for i in range(os.cpu_count()) if i != 0] # 获取除了第一个CPU核心外的所有CPU核心编号
    os.sched_setaffinity(pid, available_cores) # 设置进程的CPU亲和性, 将进程绑定到除了第一个核心以外的所有可用核心, 以实现多核并行处理

    # 1 for add 3d tiles
    # 2 for add raster relief
    # 3 for add physical field
    # 4 for add point cloud
    mode = 1
    if mode == 1:
        p3d = Parser3DTiles()
        # _3dtilesPath = "/home/program/3dsim/data/3dtiles/New-York-Manhattan-Buildings-3D-Tiles-WGS84-Quadtree-Level-14/tileset.json"
        # _3dtilesPath = "/home/program/3dsim/data/3dtiles/DA16_3D_Buildings/tileset.json"
        _3dtilesPath = "/home/program/3dsim/data/3dtiles/白膜-武汉建筑/tileset.json"
        p3d.add_data(_3dtilesPath,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
    if mode == 2:
        pr = ParserRelief()
        testPath = "/home/program/3dsim/data/rasterrelief/DTM_e36n31_TINRelief_4326.tif"
        pr.add_data(mimeType = 'GEOTIFF', path = testPath,createTime='20240226', validTime=['19950923', '20050101'])
    if mode == 3:
        ppf = ParserPhysicalField()
        hdfPath = "/home/program/3dsim/data/physicalfield/NPP_MOFTS_L1.A2024051.0124.001.2024051072405.hdf"
        ppf.add_data(mimeType = 'HDF', path = hdfPath,createTime='20240226', validTime=['19950923', '20050101'])
        netcdfPath = "/home/program/3dsim/data/physicalfield/example_1.nc"
        ppf.add_data(mimeType = 'NETCDF', path = netcdfPath,createTime='20240226', validTime=['19950923', '20050101'])
    if mode == 4:
        ppc = ParserPointcloud()
        lasPath = "/home/program/3dsim/data/pointcloud/building_01.las"
        ppc.add_data(mimeType = 'LAS', path = lasPath, createTime='20240226', validTime=['19950923', '20050101'])
        lazPath = "/home/program/3dsim/data/pointcloud/building_02.laz"
        ppc.add_data(mimeType = 'LAZ', path = lazPath, createTime='20240226', validTime=['19950923', '20050101'])
        xyzPath = "/home/program/3dsim/data/pointcloud/fake.xyz"
        ppc.add_data(mimeType = 'XYZ', path = xyzPath, createTime='20240226', validTime=['19950923', '20050101'])
    pass


start_time = time.time()
upload()
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000
print(f"Code execution time:{execution_time_ms:.2f}ms")

database "3dsim" already exists

Test: PostgreSQL available
PGSQL Connected
Connection pool size:  5
## SpatialDimension table exists
## TimeDimension table exists
## FeatureDimension table exists
## prodcution table exists
## viewpoint table exists
MongoDB Database '3dsim' created successfully!
MongoDB available!

Connected to MinIO successfully!
MinIO available!
## the 3d tile inserting ...
File 'public/3dtiles/白膜-武汉建筑/tileset.json' uploaded to '3dsim'
File 'public/3dtiles/白膜-武汉建筑/scenetree.json' uploaded to '3dsim'
File 'public/3dtiles/白膜-武汉建筑/NoLod_4.cmpt' uploaded to '3dsim'
File 'public/3dtiles/白膜-武汉建筑/NoLod_3.b3dm' uploaded to '3dsim'
File 'public/3dtiles/白膜-武汉建筑/NoLod_2.b3dm' uploaded to '3dsim'
File 'public/3dtiles/白膜-武汉建筑/NoLod_1.cmpt' uploaded to '3dsim'
File 'public/3dtiles/白膜-武汉建筑/NoLod_0.b3dm' uploaded to '3dsim'
File 'public/3dtiles/白膜-武汉建筑/tileset_3dsim.json' uploaded to '3dsim'
Folder '/home/program/3dsim/data/3dtiles/白膜-武汉建筑' uploaded to '3dsim'
Viewpoint level not fo

## 1 3DTiles

### 1.1 添加一万个3DTiles

In [ ]:
p3d = Parser3DTiles()
testPath = "/home/ydy/3dsim/DA1_3D_Buildings/DA1_3D_Buildings_Merged.json"
p3d.add_data(testPath,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d1 = Parser3DTiles()
testPath1 = "/home/ydy/3dsim/DA6-10_3D_Buildings/tileset.json"
p3d1.add_data(testPath1,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d2 = Parser3DTiles()
testPath2 = "/home/ydy/3dsim/DA20_3D_Buildings/tileset.json"
p3d2.add_data(testPath2,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d3 = Parser3DTiles()
testPath3 = "/home/ydy/3dsim/DA5_3D_Buildings/tileset.json"
p3d3.add_data(testPath3,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d4 = Parser3DTiles()
testPath4 = "/home/ydy/3dsim/DA16_3D_Buildings/tileset.json"
p3d4.add_data(testPath4,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])
p3d5 = Parser3DTiles()
testPath5 = "/home/ydy/3dsim/DA5_3D_Buildings/tileset.json"
p3d5.add_data(testPath5,featureType='Building',createTime='20230116', validTime=['19950923', '20050101'])

### 1.2 3DTiles 查询测试

In [ ]:
p3d = Parser3DTiles()
query = Query()

result = p3d.query_scene(product=['3DTiles','CityGML','OSG', 'I3S'], 
                             spatialExtent = [-180, -90, 180, 90],
                             timeSpan=['19000101', '20990101'], 
                             feature= ['Building'], viewedRange= [0,9999999],isRoot=True,query=query)

print(result)
print("size of root scene: ", len(result))

start_time = time.time()
# 将第一个结果保存为3dtiles
# p3d.save_data_to3dtiles(sceneAsset=result[0], path='./tileset.json',query=query)
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000
print(f"代码执行时间：{execution_time_ms:.2f}毫秒")

### 1.3 3DTiles 更新测试

In [ ]:
update = Update()
_update_data = {
    "productDimension": "102"
}
update_result = update.update_sceneAsset(scene_id=ObjectId("65dfede6851ccfe2824d8b52"),update_data=_update_data)

### 1.4 3DTiles 删除测试

In [ ]:
remove = Remove()
query = Query()
p3d = Parser3DTiles()
results = p3d.query_scene(product=['3DTiles','CityGML','OSG', 'I3S'], 
                             spatialExtent = [-180, -90, 180, 90],
                             timeSpan=['19000101', '20990101'], 
                             feature= ['Building'], viewedRange= [0,9999999],isRoot=True,query=query)

print("size of root scene: ", len(results))

for result in results:
    p3d.remove_scene(scene_id=result['_id'],query=query,remove=remove)

## 2 Mesh

In [ ]:
# TODO

## 3 三维点云

### 3.1 las 解析

In [ ]:
las = laspy.read("/home/program/3dsim/data/pointcloud/building_01.las")
x_min = las.header.x_min
x_max = las.header.x_max
y_min = las.header.y_min
y_max = las.header.y_max
z_min = las.header.z_min
z_max = las.header.z_max
print(x_min, x_max, y_min, y_max, z_min, z_max)

### 3.2 laz 解析

In [ ]:
laz = laspy.read("/home/program/3dsim/data/pointcloud/building_02.laz")
x_min = laz.header.x_min
x_max = laz.header.x_max
y_min = laz.header.y_min
y_max = laz.header.y_max
z_min = laz.header.z_min
z_max = laz.header.z_max
print(x_min, x_max, y_min, y_max, z_min, z_max)

### 3.3 xyz 解析

In [ ]:

with open("/home/program/3dsim/data/pointcloud/fake.xyz", 'r') as file:
    lines = file.readlines()
    data = np.array([list(map(float, line.split())) for line in lines])

def get_min_max_xyz(data):
    min_xyz = np.min(data, axis=0)
    max_xyz = np.max(data, axis=0)
    return min_xyz, max_xyz

min_xyz, max_xyz = get_min_max_xyz(data)

print("最小值 (min_x, min_y, min_z):", min_xyz)
print("最大值 (max_x, max_y, max_z):", max_xyz)


### 3.4 点云坐标变换

In [ ]:
# 读取Laz点云文件
input_laz_file = "/home/program/3dsim/data/pointcloud/building_01.las"
output_laz_file = "/home/program/3dsim/data/pointcloud/output.las"

# 读取点云数据
las = laspy.read(input_laz_file)

# 获取点云数据
xyz = np.vstack((las.x, las.y, las.z)).T
rgb = np.vstack((las.red, las.green, las.blue)).T


"""
@description: 这段代码定义了一个名为 `transform2world` 的变换矩阵，并将其应用到点云数据中。变换矩阵是一个 4x4 的矩阵，用于将点云从局部坐标系转换到世界坐标系。

具体来说，这个变换矩阵将点云的 X、Y、Z 坐标分别乘以一个缩放因子 `0.000008998719243599958`,
然后将点云沿 X 轴平移 `103.778333`，沿 Y 轴平移 `1.284444`。     
最后，变换矩阵的第四行和第四列都是 `[0, 0, 0, 1]`，表示不对点云进行旋转和投影变换。
"""
# 定义变换矩阵
transform2world = np.array([
    [0.000008998719243599958, 0.0, 0.0, 103.778333],
    [0.0, 0.000008998719243599958, 0.0, 1.284444],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
])

# 应用变换矩阵到点云数据
transformed_xyz = np.dot(xyz, transform2world[:3, :3].T) + transform2world[:3, 3]

# 创建新的Laz文件并保存点云
header = LasHeader(point_format=2, version="1.2")

# TODO laspy 2.0
#header = laspy.create(point_format=2, file_version='1.2')


header.offset = [0.0, 0.0, 0.0]  # 根据实际情况设置正确的偏移
header.scale = [0.01, 0.01, 0.01]  # 根据实际情况设置正确的比例


out_las = File(output_laz_file, mode="w", header=header)
# TODO laspy 2.0
#las.write(output_laz_file)

out_las.x = transformed_xyz[:, 0]
out_las.y = transformed_xyz[:, 1]
out_las.z = transformed_xyz[:, 2]
out_las.red = rgb[:, 0]
out_las.green = rgb[:, 1]
out_las.blue = rgb[:, 2]

# 关闭输入和输出laz文件
las.close()
out_las.close()

## 4 三维物理场数据解析

In [ ]:
def parser_physicalfield_v1():
    data=xr.open_dataset('/home/program/3dsim/data/physicalfield/example_1.nc')
    print("\nnc文件的概况:")
    print(data)
    print("\nnc文件中的变量:")
    lat_values = data.variables.keys()
    print(lat_values)

def parser_physicalfield_v2():
    nc_dataset = Dataset('/home/program/3dsim/data/physicalfield/example_1.nc', 'r')
    print("\nnc版本:")
    print(nc_dataset.data_model)
    print("\nnc文件中包含的变量:")
    print(nc_dataset.variables.keys())
    print("\nnc文件中包含的属性:")
    print(nc_dataset.ncattrs())

    lon = nc_dataset['longitude'][:]
    lat = nc_dataset['latitude'][:]
    x_min, x_max = lon.min(), lon.max()
    y_min, y_max = lat.min(), lat.max()


parser_physicalfield_v1()

### 4.1 地理坐标系下的ERA5再分析数据

In [ ]:
in_path = r"/home/program/3dsim/data/physicalfield/test1.nc"
out_path = r"/home/program/3dsim/data/physicalfield/test1.tif"

# 获取分辨率和左上角像素坐标值
rootgrp = nc.Dataset(in_path)
lon = rootgrp['longitude'][...].data
lat = rootgrp['latitude'][...].data
x_min, y_max = lon.min(), lat.max()
x_res, y_res = abs(float(lon[1]-lon[0])), abs(float(lat[1]-lat[0]))

# 仿射变换六参数
gt = (x_min, x_res, 0, y_max, 0, -y_res)

# 获取地理坐标系
crs = CRS.from_epsg(4326)
wkt = crs.to_wkt()

# 读取数据
r = rootgrp['r']
nodata = int(r._FillValue)
r = r[...].filled(nodata)

# 计算月平均相对湿度
r = r.mean(axis=0).astype(np.int32)

# 创建GeoTIFF文件并写入数据
driver = gdal.GetDriverByName('GTiff')
ds = driver.Create(out_path, r.shape[1], r.shape[0], 1, 
                gdal.GDT_Int32, options=["COMPRESS=LZW"])
ds.SetGeoTransform(gt)
ds.SetProjection(wkt)
band = ds.GetRasterBand(1)
band.WriteArray(r)
band.SetNoDataValue(nodata)

ds = band = None

### 4.2 投影坐标系下的反演数据

In [ ]:
in_path = r"/home/program/3dsim/data/physicalfield/test2.nc"
out_path = r"/home/program/3dsim/data/physicalfield/test2.tif"

# 获取分辨率和左上角像素坐标值
rootgrp = nc.Dataset(in_path)
lon = rootgrp['x'][...].data
lat = rootgrp['y'][...].data
x_min, y_max = lon.min(), lat.max()
x_res, y_res = abs(float(lon[1]-lon[0])), abs(float(lat[1]-lat[0]))

# 仿射变换六参数
gt = (x_min, x_res, 0, y_max, 0, -y_res)

# 获取地理坐标系
gr = rootgrp['polar_stereographic']
wkt = gr.crs_wkt

# 读取数据
PFR = rootgrp['PFR']
nodata = int(PFR._FillValue)
PFR = PFR[0, ...].filled(nodata)

# 创建GeoTIFF文件并写入数据
driver = gdal.GetDriverByName('GTiff')
ds = driver.Create(out_path, PFR.shape[1], PFR.shape[0], 1, 
                gdal.GDT_Int16, options=["COMPRESS=LZW"])
ds.SetGeoTransform(gt)
ds.SetProjection(wkt)
band = ds.GetRasterBand(1)
band.WriteArray(PFR)
band.SetNoDataValue(nodata)

ds = band = None

### 4.3 HDF数据解析

In [ ]:
#%% 读取HDF信息并提取数据 
HDF_FILR_URL = r"/home/program/3dsim/data/physicalfield/NPP_MOFTS_L1.A2024051.0124.001.2024051072405.hdf"
file = SD(HDF_FILR_URL)
 
print(file.info())

print("--------")
datasets_dic = file.datasets()
 
for idx,sds in enumerate(datasets_dic.keys()):
    print(idx,sds)
 
print("--------")
lat = file.select('Latitude').get() 
lon = file.select('Longitude').get()
height = file.select('Height').get()


print(lat.max(),lat.min())
print(lon.max(),lon.min())
print(height.max(),height.min())


## 5 py3dtiles 解析测试

In [ ]:
tileset = TileSet.from_file(Path("/home/program/3dsim/data/3dtiles/WHU/tileset.json"))

all_tiles = (tileset.root_tile, *tileset.root_tile.get_all_children())
for tile in all_tiles:
    if not tile.has_content():
        continue
    tile_content = tile.get_or_fetch_content(tileset.root_uri)
    print(f"The tile {tile.content_uri} has a content of {type(tile_content)} type")
    print(f"with this bounding volume: {tile.bounding_volume.to_dict()}")



## 6 RasterRelief 解析

### 6.1 重投影

In [ ]:
options = "-of GTiff -s_srs EPSG:4978 -t_srs EPSG:4326"
gdal.Warp('/home/program/3dsim/data/rasterrelief/out.tif', '/home/program/3dsim/data/rasterrelief/DTM_e36n31_TINRelief_4978.tif', options=options)

### 6.2 RasterRelief 查询测试

In [ ]:
query  = Query()
query_result = query.query_modelAsset(product=['RasterRelief','PointCloud','PhysicalField', '3DMesh'], 
         spatialExtent= [-180, -90, 180, 90],
         timeSpan= ['19000101', '20990101'], 
         feature= ['Mountain','WaterBody','Relief','SnowArea','Ocean','PhysicalField','AgriculturalArea','WaterFacility','TransportationFacility','Building','Pipeline','Courtyard','ArtificialLandform'], 
         viewedRange= [0,9999999])

### 6.3 RasterRelief 删除测试

In [ ]:
remove = Remove()
remove_result = remove.remove_modelAsset(product=['RasterRelief','PointCloud','PhysicalField', '3DMesh'], 
         spatialExtent= [-180, -90, 180, 90],
         timeSpan= ['19000101', '20990101'], 
         feature= ['Mountain','WaterBody','Relief','SnowArea','Ocean','PhysicalField','AgriculturalArea','WaterFacility','TransportationFacility','Building','Pipeline','Courtyard','ArtificialLandform'], 
         viewedRange= [0,9999999])

### 6.4 gdal2tiles 切瓦片

In [ ]:
# 将数据格式转为BYTE, 并将值域映射到0-255
options = f"-ot Byte " \
          f"-scale 0 380 0 255 " \
          
translate_options = gdal.TranslateOptions(options=options)
gdal.Translate("/home/program/3dsim/data/rasterrelief/renhuai_gpp_202001_after.tif", "/home/program/3dsim/data/rasterrelief/renhuai_gpp_202001.tif", options=translate_options)

options = { 
    'zoom':(0,8),
    'tile_size': 256,
    's_srs': "EPSG:4326",
}
gdal2tiles.generate_tiles('/home/program/3dsim/data/rasterrelief/renhuai_gpp_202001_after.tif','/home/program/3dsim/data/rasterrelief/test/', **options)

# 7 osgb转3dtiles

In [ ]:
command = 'docker run --rm -v /home/program/3dsim/data/osg:/data winner1/3dtiles:1.0 /bin/bash -c "cd /3dtiles && export LD_LIBRARY_PATH=./lib && ./target/release/_3dtile --format osgb --input /data/osgb --output /test/3dtiles"'
result = os.system(command)
print(result)